In [338]:
import folium
from folium import plugins
import requests

from folium.plugins.timeline import Timeline, TimelineSlider
from folium.utilities import JsCode
from folium.features import GeoJsonPopup

data_url = 'https://incidents.fire.ca.gov/umbraco/api/IncidentApi/GeoJsonList?year=2013'

geojson_data = requests.get(data_url).json()
print(len(geojson_data['features']))

# m = folium.Map((36.7783, -119.4179), tiles="cartodbpositron", zoom_start=6)

# folium.GeoJson(geojson_data, name="test").add_to(m)

# folium.LayerControl().add_to(m)

# timeline = Timeline(
#     geojson_data,
#     # get_interval=JsCode(
#     #     '''
#     #     function (point) {
#     #        console.log(point)
#     #        // return {
#     #        //     start: point.properties.StartedDateOnly,
#     #        //     end: point.properties.ExtinguishedDateOnly || "2025-02-26",
#     #       //  };
#     #     };
#     # '''
#     # ),
# ).add_to(m)

# GeoJsonPopup(fields=['Started'], labels=True).add_to(timeline)

# TimelineSlider(
#     start = "2013",
#     end = "2025",
#     date_options = "YYYY-MM-DD",
#     auto_play=False,
#     show_ticks=True,
#     enable_keyboard_controls=True,
#     playback_duration=30000,
# ).add_timelines(timeline).add_to(m)



159


In [346]:
import requests
import json
import xyzservices.providers as xyz
from pyproj import Transformer, transform

from bokeh.models import GeoJSONDataSource, CustomJS, Slider
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

testobj = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [-120.246601, 34.883723]
      },
      "properties": {
        "Name": "Oak Fire",
        "Updated": "2025-01-12T09:36:42Z",
        "Started": "2025-01-01T20:08:00Z",
        "AdminUnit": "Santa Barbara County Fire Department",
        "County": "Santa Barbara",
        "Location": "Tepesquet Canyon/Santa Maria Mesa Road, Santa Maria",
        "AcresBurned": 42,
        "PercentContained": 100,
        "AgencyNames": "",
        "Longitude": -120.246601,
        "Latitude": 34.883723,
        "Type": "Wildfire",
        "UniqueId": "e2f7d285-7b47-40c0-802e-b7de9e51da56",
        "Url": "https://www.fire.ca.gov/incidents/2025/1/1/oak-fire/",
        "ExtinguishedDate": "",
        "ExtinguishedDateOnly": "",
        "StartedDateOnly": "2025-01-01",
      }
    }]}

data_url = 'https://incidents.fire.ca.gov/umbraco/api/IncidentApi/GeoJsonList?year=2013'
geojson_data = requests.get(data_url).json()

for i in range(len(testobj['features'])):
    testobj['features'][i]['properties']['Color'] = ['blue', 'red'][i%2]

for i in range(len(testobj['features'])):
    coords = testobj['features'][i]['geometry']['coordinates']
    testobj['features'][i]['geometry']['coordinates'] = list(transformer.transform(coords[0], coords[1]))

geo_source = GeoJSONDataSource(geojson=json.dumps(testobj))

TOOLTIPS = [('County', '@County')]

# print(geo_source)
# print(json.dumps(testobj))

p = figure(x_range=(-14500000, -12500000), y_range=(4000000, 5000000),
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS)

p.add_tile(xyz.OpenStreetMap.Mapnik, retina=True)
p.scatter(x='x', y='y', size=15, color='Color', alpha=0.7, source=geo_source)

date_slider = Slider(value=2013,
                     start=2013,
                     end=2025
                    )
# date_slider.js_on_change("value", CustomJS(code="""
#     console.log('date_slider: value=' + this.value, this.toString())
# """))

show(date_slider)
show(p)

Loading BokehJS ...

GeoJSONDataSource(id='p8116', ...)
{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [-13385790.39294194, 4148090.769892915]}, "properties": {"Name": "Oak Fire", "Updated": "2025-01-12T09:36:42Z", "Started": "2025-01-01T20:08:00Z", "AdminUnit": "Santa Barbara County Fire Department", "County": "Santa Barbara", "Location": "Tepesquet Canyon/Santa Maria Mesa Road, Santa Maria", "AcresBurned": 42, "PercentContained": 100, "AgencyNames": "", "Longitude": -120.246601, "Latitude": 34.883723, "Type": "Wildfire", "UniqueId": "e2f7d285-7b47-40c0-802e-b7de9e51da56", "Url": "https://www.fire.ca.gov/incidents/2025/1/1/oak-fire/", "ExtinguishedDate": "", "ExtinguishedDateOnly": "", "StartedDateOnly": "2025-01-01", "Color": "blue"}}]}
